In [1]:
Threads.nthreads()

4

In [1]:
using LinearAlgebra
using Statistics
using IterativeSolvers
using Convex
using SCS
using COSMO
using NLsolve
using DifferentialEquations
using SparseArrays
using Random
using GraphPlot
using MathOptInterface

using JLD2

using LightGraphs
using PyPlot
PyPlot.svg(true);

In [2]:
n = 1
m = 13
# e = 50

# g = complete_bipartite_graph(n, m)

g = LightGraphs.grid([n, m], periodic=true)
N = nv(g)
# add_edge!(g, (1, 2))
# add_edge!(g, (3, 2))

# balanced power flows/natural frequencies
ω = randn(nv(g))
ω .-= mean(ω)
ω = ω./std(ω);

is_connected(g)

true

In [3]:
function optimize_graph(g, K, ω; max_iter=200000, iterations=10000)
    N = nv(g)

    E = incidence_matrix(g; oriented=true)

    # coupling constants
    B = (K/mean(LightGraphs.degree(g)))*ones(ne(g));

    # laplacian
    L = Symmetric(E*diagm(0=>B)*E')

    # u, Q = eigen(L)

    L_sp = dropzeros(sparse(L))

    function f_steady_state(F, δ)
        F .= ω/nv(g) - E*(B.*sin.(E'*δ))
    end

    # initial condition
    δ0 = 1e-2randn(nv(g))
    sln = nlsolve(f_steady_state, δ0; autodiff=:forward, iterations=iterations)

    # order parameter at the fixed point
    R0 = abs2(mean(exp.(1im*sln.zero)))

    # check by solving ODE
    function f_kuramoto(du, u, p, t)
        f_steady_state(du, u)
    end

    # Noisy extension
    δbar = sln.zero
    
    if converged(sln)
        # construct S matrix
        h = complete_graph(nv(g))
        Eh = incidence_matrix(h; oriented=true)
        cosδbar = cos.(Eh'*δbar)
        # cosδbar = ones(ne(h))

        # Laplacian of the corresponding complete graph
        S = Symmetric(-Eh*diagm(0=>cosδbar)*Eh'/nv(g)^2)

        # weighted Laplacian expanded around fixed point
        B_fp = cos.(E'*δbar).*B
        L_fp = Symmetric(E*diagm(0 => B_fp)*E')

        # optimize!
        Eq = Convex.Semidefinite(nv(g))
        C = Convex.Semidefinite(nv(g))

        problem = maximize(tr(S*Eq), [
                L_fp*Eq + Eq*L_fp == C, 
                Eq*ones(N) == zeros(N),
        #         sum(Eq) == 0.0,
                diag(C, 0) == 1.0,
        #         tr(C) == nv(g),
                ])

        Convex.solve!(problem, () -> COSMO.Optimizer(verbose=false, 
                eps_abs=1e-7, eps_rel=1e-7, max_iter=max_iter))

        @show R0, converged(sln)
        @show problem.optval, problem.status
    end
    
    if converged(sln) && (problem.status == MathOptInterface.OPTIMAL)
        C.value, Eq.value, S, L_fp, R0, problem.optval, δbar, B, E
    else
        [NaN], [NaN], [NaN], [NaN], NaN, NaN, [NaN], B, E
    end
end

optimize_graph (generic function with 1 method)

In [9]:
K = 2.0
Copt, Eopt, S, L_fp, R0, optval, δbar, B, E = optimize_graph(g, K, ω)

C_unif = ((1 + 1/nv(g))I - ones(nv(g), nv(g))/nv(g))

(R0, converged(sln)) = (0.9757834675722112, true)
(problem.optval, problem.status) = (-0.12423959572390249, MathOptInterface.OPTIMAL)


13×13 Matrix{Float64}:
  1.0        -0.0769231  -0.0769231  …  -0.0769231  -0.0769231  -0.0769231
 -0.0769231   1.0        -0.0769231     -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231   1.0           -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231  …  -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231  -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231  …   1.0        -0.0769231  -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231   1.0        -0.0769231
 -0.0769231  -0.0769231  -0.0769231     -0.0769231  -0.0769231   1.0

In [15]:
Ks = LinRange(0.01, 0.5, 51)

Cvals = []
Copts = []
optvals = []
R0s = []
δbars = []

cur_g = g
for (i, k) in enumerate(Ks)
    Copt, Eopt, S, L_fp, R0, optval, δbar = optimize_graph(cur_g, k, ω; iterations=5000, max_iter=200000)
    
    push!(Copts, Copt)
    push!(Cvals, sum(abs.(Copt))/N^2)
    push!(optvals, optval)
    push!(R0s, R0)
    push!(δbars, δbar)
    
    @show i, k, Cvals[end]
end

(i, k, Cvals[end]) = (1, 0.01, NaN)
(i, k, Cvals[end]) = (2, 0.019799999999999998, NaN)
(i, k, Cvals[end]) = (3, 0.0296, NaN)
(i, k, Cvals[end]) = (4, 0.0394, NaN)
(i, k, Cvals[end]) = (5, 0.0492, NaN)
(i, k, Cvals[end]) = (6, 0.059000000000000004, NaN)
(i, k, Cvals[end]) = (7, 0.0688, NaN)
(i, k, Cvals[end]) = (8, 0.0786, NaN)
(i, k, Cvals[end]) = (9, 0.0884, NaN)
(i, k, Cvals[end]) = (10, 0.0982, NaN)
(i, k, Cvals[end]) = (11, 0.10800000000000001, NaN)
(i, k, Cvals[end]) = (12, 0.1178, NaN)
(i, k, Cvals[end]) = (13, 0.1276, NaN)
(i, k, Cvals[end]) = (14, 0.1374, NaN)
(i, k, Cvals[end]) = (15, 0.14720000000000003, NaN)
(i, k, Cvals[end]) = (16, 0.157, NaN)
(i, k, Cvals[end]) = (17, 0.1668, NaN)
(i, k, Cvals[end]) = (18, 0.1766, NaN)
(i, k, Cvals[end]) = (19, 0.18639999999999998, NaN)
(i, k, Cvals[end]) = (20, 0.1962, NaN)
(i, k, Cvals[end]) = (21, 0.20600000000000002, NaN)
(i, k, Cvals[end]) = (22, 0.2158, NaN)
(i, k, Cvals[end]) = (23, 0.2256, NaN)
(i, k, Cvals[end]) = (24, 0.2354, N

In [30]:
# simulate time series and check optimization

# find a particular realization of the noise input
function G_from_C(C)
    U, Σ, V = svd(C)
    U*diagm(0 => sqrt.(Σ))*U'
end

function solve_sde_v2(σ, δ0, G, B; tspan=(0.0, 3000.0), dt=0.01, saveat=nothing)
    function Rsqr_from_u(u)
        abs(mean(exp.(1im*u)))^2
    end
    
    function Rsqr_grad(u)
        # gradient of R^2
        n = length(u)
        J = zeros(n)
        
        # can we make this nicer?
        for i=1:n
            J[i] = 2/n^2 * sum(sin.(u .- u[i]))
        end
        
        J
    end
    
    function f_steady_state(F, δ)
        F .= ω/nv(g) - E*(B.*sin.(E'*δ))
    end
    
    # solve kuramoto model *and* automatically get time average of the order parameter
    function f_kuramoto(du, u, p, t)
        dδ = @view du[1:end-1]
        f_steady_state(dδ, u[1:end-1])
        
        # time average y(t) = 1/t\int_0^t R^2(t) dt of the order parameter satisfies
        # dy/dt = -y/t + R^2(t)/t
        
        if t > 0
            # solve ode
            y = u[end]
            Rsqr = Rsqr_from_u(u[1:end-1])
            
            du[end] = -y/t + Rsqr/t
        else
            # use formula for y'(0) = 1/2 (R^2)'(0) = 1/2 grad(R^2)(0)'*du(0)
            J = Rsqr_grad(u[1:end-1])
            du[end] = 0.5J'*du[1:end-1]
        end
    end
    
    # σ is the noise strength (multiplies the noise matrix)
    function g_kuramoto(du, u, p, t)
        du[1:end-1,:] .= 1/sqrt(2)*σ*G
        
        # no noise on the time average
        du[end,:] .= 0.0
    end
    
    n, m = size(G)
    
    Rsqr0 = Rsqr_from_u(δ0)
    u0 = vcat(δ0, [Rsqr0])
    prob = SDEProblem(f_kuramoto, g_kuramoto, u0, tspan; noise_rate_prototype=zeros(n+1, m))
    sdesln = solve(prob, LambaEM(), dt=dt, saveat=saveat)
end

function cumtrapz_avg(t::T, Y::T) where {T <: AbstractVector}
    # Estimates the cumulative time average integral 1/T ∫₀ᵀ f(t) dt using the trapezoid rule
    # where time points are in t and corresponding samples of f are in Y
    
    # Check matching vector length
    @assert length(t) == length(Y)
    
    # Initialize Output
    out = similar(t)
    out[1] = 0.0
    # Iterate over arrays
    for i in 2:length(t)
        out[i] = out[i-1] + 0.5*(t[i] - t[i-1])*(Y[i] + Y[i-1])
    end
    out[2:end] ./= (t[2:end] .- t[1])
    out[1] = out[2]

    out
end

function Rsqr_from_sde(sdesln; cut=true)
    # numerically integrate and average
    Rsqrs = [abs(mean(exp.(1im*u)))^2 for u in sdesln.u]
end

Rsqr_from_sde (generic function with 1 method)

In [31]:
# find first Copt that is not NaN
Copt_first = Copts[end]
idx_first = length(Copts)

for (i, Copt) in enumerate(Copts)
    if !isnan(Copt[1])
        Copt_first = Copt
        idx_first = i
        break
    end
end

idx_first

30

In [32]:
Rsqrs_opt = Float64[]
Ks_opt = Float64[]
Rsqrs_uni = Float64[]
Ks_uni = Float64[]
Rsqrs_no = Float64[]
Ks_no = Float64[]

σ = 0.5
G_unif = G_from_C(C_unif)

tmax = 2000000.0

for (i, (k, δbar, Copt)) in enumerate(zip(Ks, δbars, Copts))
    if isnan(Copt[1])
        # pick the first optimal noise matrix that did converge
        Copt = Copt_first
        
        # if this one didn't converge, start from zeros
        if isnan(δbar[1])
            δbar = zeros(nv(g))
        end
    end
    
    B = (k/mean(LightGraphs.degree(g)))*ones(ne(g));
    G = G_from_C(Copt)
    
    for j=1:3
        if j == 1
            sdesln_v21 = solve_sde_v2(σ, δbar, G, B; tspan=(0.0, tmax), saveat=[tmax])
            push!(Rsqrs_opt, sdesln_v21.u[end][end])
            push!(Ks_opt, k)
        elseif j == 2
            sdesln_v22 = solve_sde_v2(σ, δbar, G_unif, B; tspan=(0.0, tmax), saveat=[tmax])
            push!(Rsqrs_uni, sdesln_v22.u[end][end])
            push!(Ks_uni, k)
        elseif j == 3
            sdesln_v23 = solve_sde_v2(0.0, δbar, G_unif, B; tspan=(0.0, tmax), saveat=[tmax])
            push!(Rsqrs_no, sdesln_v23.u[end][end])
            push!(Ks_no, k)
        end
    end
        
    @show i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]
end

(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (1, 0.08047162491969573, 0.07929753427072074, 0.08610712678995848)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (2, 0.07758229996779875, 0.07621113450013249, 0.0793417370505448)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (3, 0.07881674268670616, 0.07709414199991117, 0.07752158974999895)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (4, 0.07780372000830933, 0.07710874872749267, 0.09669124776693908)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (5, 0.07916080821998306, 0.0824386251431388, 0.08931377459320922)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (6, 0.07622372028055388, 0.0830196531317072, 0.08565575515163182)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (7, 0.07882732966299409, 0.09944201119084518, 0.08288439986149368)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (8, 0.07921057376173764, 0.09809311694602739, 0.08320803037549065)
(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_n

┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331


(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (38, 0.2627609682846133, 0.295723005148226, 0.4074718202301692)


┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331


(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (39, 0.2608898318803844, 0.31533363813216236, 0.4324824371328376)


...

(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (49, 0.28632211727080303, 0.3050379613972049, 0.6181088150492179)


┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331


(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (50, 0.29273391034434754, 0.31594656146425315, 0.6316629800404084)


┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331
┌ Warning: Interrupted. Larger maxiters is needed.
└ @ SciMLBase /home/hmr1/.julia/packages/SciMLBase/x3z0g/src/integrator_interface.jl:331


(i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]) = (51, 0.29582427282604223, 0.324205539387165, 0.6445828509059691)


In [ ]:
fig, axs = subplots(1, 2, figsize=(8, 3))

ax = axs[1]
ax.plot(Ks_no, Rsqrs_no, "k--", label="no noise")
ax.plot(Ks, R0s, "k", label="noise-free")
ax.plot(Ks_uni, Rsqrs_uni, label="uniform noise")
ax.plot(Ks_opt, Rsqrs_opt, label="optimized noise")
# ax.plot(Ks, R0s .+ 0.5*(σ^2)*optvals, "C1", label="optimized noise")


# ax.plot(Ks, R0s, label="noise-free")
# ax.plot(Ks, R0s .+ 0.5*(0.2^2)*optvals, label="optimized noise - predicted")

ax.set_xlabel("K")
ax.set_ylabel("⟨R²⟩")
ax.set_title("numerics")
ax.legend()

ax.legend()

ax = axs[2]
ax.plot(Ks, R0s, label="noise-free")
ax.plot(Ks, R0s .+ 0.5*(σ^2)*optvals, label="optimized noise")
ax.set_xlabel("K")
ax.set_ylabel("⟨R²⟩ = ⟨R₀²⟩ + (1/2) σ² tr(HE)")
ax.set_title("theory")

ax.legend()

fig.tight_layout()

In [ ]:
jldsave("data/chain_Ks_n_$(n)_m_$(m)_sigma_$(σ).jld2"; g, Copts, Cvals, R0s, optvals, Ks, Rsqrs_uni, Rsqrs_opt, Rsqrs_no, σ)

In [ ]:
σs = LinRange(0, 5, 51)
Kfix = 0.2

In [ ]:
Rsqrs_opt = Float64[]
Rsqrs_uni = Float64[]
Rsqrs_no = Float64[]

G_unif = G_from_C(C_unif)

tmax = 200000.0

δbar = zeros(nv(g))
for (i, σ) in enumerate(σs)
    Copt = Copt_first
    
    B = (Kfix/mean(LightGraphs.degree(g)))*ones(ne(g));
    G = G_from_C(Copt)
    
    for j=1:3
        if j == 1
            sdesln_v21 = solve_sde_v2(σ, δbar, G, B; tspan=(0.0, tmax), saveat=[tmax])
            push!(Rsqrs_opt, sdesln_v21.u[end][end])
        elseif j == 2
            sdesln_v22 = solve_sde_v2(σ, δbar, G_unif, B; tspan=(0.0, tmax), saveat=[tmax])
            push!(Rsqrs_uni, sdesln_v22.u[end][end])
        elseif (j == 3) && (i == 1)
            sdesln_v23 = solve_sde_v2(0.0, δbar, G_unif, B; tspan=(0.0, tmax), saveat=[tmax])
            push!(Rsqrs_no, sdesln_v23.u[end][end])
        end
    end
        
    @show i, Rsqrs_uni[end], Rsqrs_opt[end], Rsqrs_no[end]
end

In [ ]:
fig, axs = subplots(1, 2, figsize=(8, 3))

ax = axs[1]
ax.axhline(Rsqrs_no[1], ls="--", c="k", label="no noise")
# ax.plot(σs, R0s, "k", label="noise-free")
ax.plot(σs, Rsqrs_uni, label="uniform noise")
ax.plot(σs, Rsqrs_opt, label="optimized noise")
# ax.plot(Ks, R0s .+ 0.5*(σ^2)*optvals, "C1", label="optimized noise")


# ax.plot(Ks, R0s, label="noise-free")
# ax.plot(Ks, R0s .+ 0.5*(0.2^2)*optvals, label="optimized noise - predicted")

ax.set_xlabel("σ")
ax.set_ylabel("⟨R²⟩")
ax.set_title("numerics")
ax.legend()

ax.legend()

fig.tight_layout()

In [ ]:
jldsave("data/chain_sigmas_n_$(n)_m_$(m).jld2"; g, Copt_first, Kfix, Rsqrs_uni, Rsqrs_opt, Rsqrs_no, σs)